In [2]:
import flywheel
import os
import datetime
from dateutil.tz import tzutc

fw = flywheel.Client()
gear = fw.lookup('gears/wscore-ct-heatmap/0.2.1_0.2.0')
collection_id = '60a4032befa949862afeafbb'

local_dir = '/home/will/Projects/PVS/wscore_data'
os.makedirs(local_dir, exist_ok=True)

In [3]:
# Get all the sessions within the collection.
sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)]

In [4]:
for session in sessions:
    sub_label = fw.get(session.parents['subject']).label.replace('_','x')
    ses_dir = os.path.join(local_dir, sub_label, session.label)
    os.makedirs(ses_dir, exist_ok=True)
    
    for analysis in session.analyses:
        if analysis.label.startswith('wscore_2021-05-26') and analysis.job['state']=='complete' and analysis.files:
            file = [f for f in analysis.files if f.name.endswith('wscores.csv')]
            
            try:
                analysis.download_file(file[0].name, os.path.join(ses_dir, file[0].name))
            except IndexError as e:
                print(session.label)